In [1]:
import duckdb
from tqdm.auto import tqdm 

connection_source = duckdb.connect(database="../dbt/database_name.duckdb", read_only = True)
connection_destination = duckdb.connect(database="../dbt/metabase.duckdb")

In [2]:
# Schéma source et destination
source_schema = "main_marts"
destination_schema = "main"


In [3]:
public_tables = ["marts_public_establishments_morphology", "marts_public_establishments_morphology_unpivoted"] 

analysis_tables = ["marts_analysis_exit_flow_ff23_lovac"]

common_tables = ["marts_common_cities", "marts_common_morphology"]

production_tables = ["marts_stats_monthly_global",
          "marts_production_housing",
          "marts_production_establishments",
          "marts_production_owners",
          "marts_production_groups",
          "marts_production_users",
          "marts_production_campaigns"    
]

join_tables = [
    "marts_production_join_campaigns_housing",
    "marts_production_join_establishment_cities",
    "marts_production_join_housing_groups",
    "marts_production_join_owner_housing",
    "marts_production_join_establishment_housing"
]

In [4]:
filters = []

In [5]:
translation_mapping ={
    "marts_production_join_campaigns_housing" : "join_campaigns_housing",
    "marts_production_join_establishment_cities": "join_establishment_cities",
    "marts_production_join_housing_groups": "join_housing_groups",
    "marts_production_join_owner_housing": "join_owner_housing",
    "marts_production_join_establishment_housing": "join_establishment_housing",
    "marts_common_cities": "cities_zonage_2024",
    #"marts_common_morphology": "infra_municipalities_morphology",
    #"marts_production_campaigns": "prod_campaigns",
    #"marts_production_establishments": "prod_establishments",
    #"marts_production_groups": "prod_groups",
    #"marts_production_housing": "prod_housing",
    #"marts_production_owners": "prod_owners",
    #"marts_production_users": "prod_users",
    #"marts_public_establishments_morphology": "stats_establishments_morphology_annuals",
    #"marts_public_establishments_morphology_unpivoted": "stats_establishments_morphology_stocks",
    #"marts_stats_monthly_global": "stats_activity_monthly",
}

In [6]:
tables = production_tables + join_tables + common_tables

if filters:
    tables = filters

In [7]:
import numpy as np

In [8]:
# Boucle sur chaque table pour transférer les données
for table in tqdm(tables, desc="Transfert des tables"):
    # Lecture des données de la table source
    query_read = f'SELECT * FROM {source_schema}.{table}'
    df = connection_source.execute(query_read).fetchdf()
    df = df.replace({None: np.nan})

    # Ecriture des données dans la table de destination
    destionation_table = translation_mapping.get(table, table)
    query_write = f'CREATE OR REPLACE TABLE {destination_schema}.{destionation_table} AS SELECT * FROM df'
    connection_destination.execute(query_write)

# Fermeture des connexions
connection_source.close()
connection_destination.close()

Transfert des tables:   0%|          | 0/14 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))